<a href="https://colab.research.google.com/github/Goluyadavu/Computer-Vision-Lip-Reading-2.0/blob/main/Samurai_object%20_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Clone SAMURAI repository
!git clone https://github.com/ashrafulwork/samurai_implementation.git samurai
%cd samurai

# Install required packages
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -e .
!pip install -e ".[notebooks]"
!pip install numpy opencv-python matplotlib tikzplotlib jpeg4py lmdb pandas scipy loguru

Cloning into 'samurai'...
remote: Enumerating objects: 913, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 913 (delta 53), reused 22 (delta 22), pack-reused 843 (from 3)
Receiving objects: 100% (913/913), 69.08 MiB | 16.64 MiB/s, done.
Resolving deltas: 100% (203/203), done.
/content/samurai
Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
!chmod +x /content/samurai/checkpoints/download_ckpts.sh
!cd checkpoints && ./download_ckpts.sh

--2025-03-20 12:57:41--  https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_tiny.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.102, 108.157.254.15, 108.157.254.121, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156008466 (149M) [application/vnd.snesdev-page-table]
Saving to: ‘sam2.1_hiera_tiny.pt.1’

sam2.1_hiera_tiny.p 100%[===================>] 148.78M   208MB/s    in 0.7s    

2025-03-20 12:57:42 (208 MB/s) - ‘sam2.1_hiera_tiny.pt.1’ saved [156008466/156008466]

--2025-03-20 12:57:42--  https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_small.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.102, 108.157.254.15, 108.157.254.121, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 

In [7]:
import cv2

video_path = "/content/happy.mp4"  # Change this to your video path
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video")
else:
    print("Video loaded successfully!")

cap.release()

Video loaded successfully!


In [8]:
!pip install decord ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 49.2 MB/s eta 0:00:00


In [9]:
import cv2
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Change model as needed

video_path = "/content/happy.mp4"  # Change this to your video path
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
cap.release()

if not ret:
    print("Error loading video")
    exit()

results = model(frame)

OBJECT_CLASS_ID = 2  # Change this to the desired object class ID

bbox = None
for detection in results[0].boxes.data:
    x1, y1, x2, y2, conf, cls = detection.tolist()
    if int(cls) == OBJECT_CLASS_ID:
        bbox = (int(x1), int(y1), int(x2), int(y2))
        break

if bbox is None:
    print("No object detected in the first frame.")
    exit()

# Save the bounding box
with open("bbox.txt", "w") as f:
    f.write(f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}\n")

print(f"Bounding box saved: {bbox}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 308MB/s]



0: 384x640 2 persons, 1 car, 1 skateboard, 361.5ms
Speed: 38.3ms preprocess, 361.5ms inference, 457.6ms postprocess per image at shape (1, 3, 384, 640)
Bounding box saved: (176, 294, 229, 328)


In [14]:
!pip install iopath fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31528 sha256=78cbecf8eda11ea75e31764a8e7cb484d7d499ae226ef8b7d04a3f841656693e
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=3c2f7fa0a94bf37edf8cfb0eb7975cd2ddf5c97098913bf65ecfe07d3f95acab
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
Successfully built iopath fvcore


In [15]:
!ls /content/samurai/sam2/sam2/modeling/backbones/

hieradet.py  image_encoder.py  __init__.py  __pycache__  utils.py


In [17]:
!mkdir -p /content/samurai/checkpoints

In [18]:
!wget -O /content/samurai/checkpoints/sam2.1_hiera_base_plus.pt "https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_base_plus.pt"

--2025-03-20 13:22:50--  https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_base_plus.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.33.183.29, 13.33.183.121, 13.33.183.115, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.33.183.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 323606802 (309M) [application/vnd.snesdev-page-table]
Saving to: ‘/content/samurai/checkpoints/sam2.1_hiera_base_plus.pt’

/content/samurai/ch 100%[===================>] 308.62M  25.8MB/s    in 6.7s    

2025-03-20 13:22:57 (46.1 MB/s) - ‘/content/samurai/checkpoints/sam2.1_hiera_base_plus.pt’ saved [323606802/323606802]



In [19]:
!mkdir -p /content/sam2/checkpoints/
!mv /content/samurai/checkpoints/sam2.1_hiera_base_plus.pt /content/sam2/checkpoints/

In [20]:
 !python '/content/samurai/scripts/demo.py' \
  --video_path '/content/happy.mp4' \
  --txt_path '/content/samurai/bbox.txt' \
  --video_output_path '/content/output.mp4'  # Change this to your desired output path

/content/samurai/./sam2/sam2/modeling/sam/transformer.py:23: UserWarning: Flash Attention is disabled as it requires a GPU with Ampere (8.0) CUDA capability.
  OLD_GPU, USE_FLASH_ATTN, MATH_KERNEL_ON = get_sdpa_settings()
SAMURAI mode: True
Traceback (most recent call last):
  File "/content/samurai/scripts/demo.py", line 122, in <module>
    main(args)
  File "/content/samurai/scripts/demo.py", line 44, in main
    predictor = build_sam2_video_predictor(model_cfg, args.model_path, device="cuda:0")
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/samurai/./sam2/sam2/build_sam.py", line 130, in build_sam2_video_predictor
    _load_checkpoint(model, ckpt_path)
  File "/content/samurai/./sam2/sam2/build_sam.py", line 159, in _load_checkpoint
    sd = torch.load(ckpt_path, map_location="cpu", weights_only=True)["model"]
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-package

In [21]:
!python '/content/samurai/scripts/demo.py' \
  --video_path '/content/happy.mp4' \
  --txt_path '/content/samurai/bbox.txt' \
  --model_path '/content/sam2/checkpoints/sam2.1_hiera_base_plus.pt' \
  --video_output_path '/content/output.mp4'

/content/samurai/./sam2/sam2/modeling/sam/transformer.py:23: UserWarning: Flash Attention is disabled as it requires a GPU with Ampere (8.0) CUDA capability.
  OLD_GPU, USE_FLASH_ATTN, MATH_KERNEL_ON = get_sdpa_settings()
SAMURAI mode: True
^C


In [12]:
!pip install hydra-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.8 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144555 sha256=860ae38f110c8b8cd4381d520a9370f7825f2a7c2dbb85014d97c5f3a8a382e5
  Stored in directory: /root/.cache/pip/wheels/1a/97/32/461f837398029ad76911109f07047fde1d7b661a147c7c56d1
Successfully built antlr4-python3-runtime
